<a href="https://colab.research.google.com/github/Giant-Leap-ai/ATF-Metric-Mining/blob/main/ATF_metricmining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1

In [ ]:
!pip -q install gspread pandas openai tqdm tenacity


In [ ]:
import re, json, time, math
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm

import gspread
from google.colab import auth
from google.auth import default as google_auth_default

from openai import OpenAI


In [ ]:
# --- Google auth ---
auth.authenticate_user()
creds, _ = google_auth_default()
gc = gspread.authorize(creds)

SPREADSHEET_NAME = "ATF Metric Tracker"
ss = gc.open(SPREADSHEET_NAME)

people_ws = ss.worksheet("People")
people_df = pd.DataFrame(people_ws.get_all_records())

# Load all EmailEventsYYYY tabs
email_sheets = [ws for ws in ss.worksheets() if ws.title.startswith("EmailEvents")]

email_dfs = []
for ws in email_sheets:
    df = pd.DataFrame(ws.get_all_records())
    if df.empty:
        continue
    df["__source_sheet"] = ws.title
    df["Year"] = int(ws.title.replace("EmailEvents", ""))
    email_dfs.append(df)

email_df = pd.concat(email_dfs, ignore_index=True)

print("People rows:", len(people_df))
print("Email rows:", len(email_df))
print("Years:", sorted(email_df["Year"].unique()))


People rows: 4566
Email rows: 92433
Years: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]


In [ ]:
from google.colab import userdata
import os

# Map Colab Secret "OPENAI" → OpenAI SDK expected env var
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI")

# Sanity check (should be True)
print("OPENAI_API_KEY loaded:", bool(os.environ.get("OPENAI_API_KEY")))


OPENAI_API_KEY loaded: True


In [ ]:
AFFILIATION_LABELS = [
    "Media",
    "Press/Media",
    "Government",
    "Academic",
    "Event/conference",
    "Webinar",
    "Internal",
    "DC",
    "Summit",
    "Insurance",
    "Industry",
]

INTERNAL_DOMAINS = {"afterthefireusa.org"}

KW = {
    "Webinar": ["webinar"],
    "Summit": ["summit"],
    "Event/conference": ["conference", "panel", "keynote", "convening", "symposium", "event"],
    "Press/Media": ["press", "media request", "interview", "journalist", "reporter"],
    "Insurance": ["insurance", "carrier", "claims", "underwriting"],
    "DC": ["dc", "capitol hill", "congress", "senate", "house of representatives"],
}

ROLE_WEIGHT = {"from": 1.0, "to": 0.7, "cc": 0.4, "bcc": 0.3}

EMAIL_SPLIT_RE = re.compile(r"\s*;\s*")

PHONE_RE = re.compile(
    r"(?:(?:\+?1\s*)?\(?\d{3}\)?[\s.\-]*\d{3}[\s.\-]*\d{4}"
    r"(?:\s*(?:x|ext\.?|extension)\s*\d{1,6})?)",
    re.IGNORECASE
)

def split_emails(val):
    if not val:
        return []
    return [e.strip().lower() for e in EMAIL_SPLIT_RE.split(str(val)) if "@" in e]

def email_domain(addr):
    addr = (addr or "").lower()
    return addr.split("@")[-1] if "@" in addr else ""

def extract_phones(text):
    if not text:
        return []
    phones = PHONE_RE.findall(str(text))
    uniq = []
    for p in phones:
        p = re.sub(r"\s+", " ", p).strip()
        if p not in uniq:
            uniq.append(p)
    return uniq[:5]

def keyword_hits(text):
    t = (text or "").lower()
    return {k: sum(1 for w in words if w in t) for k, words in KW.items()}


# Expand emails into participant-level evidence rows

In [ ]:
# Ensure stable email row ID
if "__email_row_id" not in email_df.columns:
    email_df["__email_row_id"] = email_df.index.astype(int)


In [ ]:
# =========================================================
# BLOCK 2.5 — BUILD PERSON-LEVEL EMAIL METRICS
# =========================================================

INTERNAL_DOMAINS = {"afterthefireusa.org"}

rows = []

for _, r in email_df.iterrows():
    year = r["Year"]

    participants = []
    if r.get("FromEmail"):
        participants.append(r["FromEmail"].lower().strip())
    participants += split_emails(r.get("ToEmails", ""))
    participants += split_emails(r.get("CCEmails", ""))
    participants += split_emails(r.get("BCCEmails", ""))

    for email in set(participants):
        domain = email_domain(email)
        is_internal = domain in INTERNAL_DOMAINS

        rows.append({
            "Email": email,
            "IsInternal": int(is_internal)
        })

metrics_df = (
    pd.DataFrame(rows)
    .groupby("Email")
    .agg(
        InternalEmailCount=("IsInternal", "sum"),
        TotalEmailCount=("IsInternal", "count")
    )
    .reset_index()
)

metrics_df["ExternalEmailCount"] = (
    metrics_df["TotalEmailCount"] - metrics_df["InternalEmailCount"]
)

metrics_df["InternalEmailShare"] = (
    metrics_df["InternalEmailCount"] /
    metrics_df["TotalEmailCount"].replace(0, pd.NA)
).fillna(0.0)

person_df = metrics_df

print("Person metrics rows:", len(person_df))
person_df.head()


Person metrics rows: 13439


,Email,InternalEmailCount,TotalEmailCount,ExternalEmailCount,InternalEmailShare
0,'chelene.lopez@stjoe.org',0,1,1,0.0
1,10077664@k12.hi.us,0,2,2,0.0
2,1carolina.spence@gmail.com,0,2,2,0.0
3,1samanthastokes@gmail.com,0,4,4,0.0
4,1sarahkatz@gmail.com,0,1,1,0.0


In [ ]:
# =================================================
# BLOCK 3 — BUILD PARTICIPANT-LEVEL EVIDENCE (ROLE-FREE)
# =================================================

# Ensure stable email row ID
if "__email_row_id" not in email_df.columns:
    email_df["__email_row_id"] = email_df.index.astype(int)

def iter_participants(row):
    if row.get("FromEmail"):
        yield (row["FromEmail"].strip().lower(), "from")
    for field, role in [("ToEmails","to"), ("CCEmails","cc"), ("BCCEmails","bcc")]:
        for e in split_emails(row.get(field, "")):
            yield (e, role)

evidence_rows = []

for _, r in tqdm(email_df.iterrows(), total=len(email_df)):
    email_row_id = int(r["__email_row_id"])

    subject = str(r.get("Subject","") or "")[:200]
    body = str(r.get("EmailBodies","") or "")
    thread = str(r.get("ThreadBodies","") or "")

    # Text preserved purely for context
    text_for_signals = subject + "\n" + body[:1200]

    # Phone presence only (contextual, not ownership)
    phones = extract_phones(body) or extract_phones(thread[:2000])
    has_phone = 1 if phones else 0

    # Lightweight keyword hits (routing hints only; optional downstream)
    hits = keyword_hits(text_for_signals)

    # Domains of all participants in the email
    domains = set()
    for addr in (
        [r.get("FromEmail")] +
        split_emails(r.get("ToEmails","")) +
        split_emails(r.get("CCEmails","")) +
        split_emails(r.get("BCCEmails",""))
    ):
        d = email_domain(addr)
        if d:
            domains.add(d)

    # Evidence richness = sampling priority ONLY
    richness = (
        min(len(body), 2000) / 2000 +
        (0.4 if has_phone else 0) +
        0.2 * sum(1 for v in hits.values() if v > 0)
    )

    for p_email, role in iter_participants(r):
        evidence_rows.append({
            "__email_row_id": email_row_id,
            "ParticipantEmail": p_email,
            "EmailRole": role,                  # from / to / cc / bcc
            "EmailRoleWeight": ROLE_WEIGHT.get(role, 0.5),
            "Year": int(r["Year"]),
            "Subject": subject,
            "BodyExcerpt": body[:600],
            "Domains": ";".join(sorted(domains)),
            "HasPhone": has_phone,
            "Richness": richness,
            **{f"Hit_{k}": v for k, v in hits.items()}
        })

evidence_df = pd.DataFrame(evidence_rows)

print("✅ Block 3 complete (role-free)")
print("Evidence rows:", len(evidence_df))
print("Unique participant emails:", evidence_df["ParticipantEmail"].nunique())
print("Evidence rows with phone presence:", int(evidence_df["HasPhone"].sum()))


100%|██████████| 92433/92433 [00:16<00:00, 5442.08it/s]


✅ Block 3 complete (role-free)
Evidence rows: 311760
Unique participant emails: 13439
Evidence rows with phone presence: 130910


In [ ]:
MAX_EVIDENCE_PER_PERSON = 30
MAX_PER_YEAR = 5

sampled_rows = []

for email, g in tqdm(evidence_df.groupby("ParticipantEmail")):
    g = g.sort_values("Richness", ascending=False)

    by_year = defaultdict(int)
    picked = []

    for _, r in g.iterrows():
        y = r["Year"]
        if by_year[y] >= MAX_PER_YEAR:
            continue
        picked.append(r)
        by_year[y] += 1
        if len(picked) >= MAX_EVIDENCE_PER_PERSON:
            break

    sampled_rows.extend(picked)

sampled_df = pd.DataFrame(sampled_rows)
print("Sampled evidence:", sampled_df.shape)


100%|██████████| 13439/13439 [00:11<00:00, 1124.51it/s]


Sampled evidence: (52220, 16)


In [ ]:
# =========================================================
# BLOCK 4 — SAMPLE EVIDENCE PER PERSON (LLM SAFE)
# =========================================================

MAX_EVIDENCE_PER_PERSON = 30

# Ensure required columns exist
required_cols = {"ParticipantEmail", "Richness"}
missing = required_cols - set(evidence_df.columns)
if missing:
    raise ValueError(f"evidence_df missing columns: {missing}")

# Sort by richness so we keep the most informative rows
evidence_df_sorted = evidence_df.sort_values(
    by="Richness",
    ascending=False
)

# Take top-N evidence rows per participant
sampled_people_df = (
    evidence_df_sorted
    .groupby("ParticipantEmail", group_keys=False)
    .head(MAX_EVIDENCE_PER_PERSON)
    .reset_index(drop=True)
)

print("Sampled evidence rows:", len(sampled_people_df))
print(
    "People with sampled evidence:",
    sampled_people_df["ParticipantEmail"].nunique()
)

sampled_people_df.head()


Sampled evidence rows: 87889
People with sampled evidence: 13439


,__email_row_id,ParticipantEmail,EmailRole,EmailRoleWeight,Year,Subject,BodyExcerpt,Domains,HasPhone,Richness,Hit_Webinar,Hit_Summit,Hit_Event/conference,Hit_Press/Media,Hit_Insurance,Hit_DC
0,75433,info@cclr.org,from,1.0,2020,,"View this email in your browser Each year, CCL...",cclr.org;rebuildnorthbay.org,1,2.2000,1,1,1,0,0,1
1,75433,jennifer@rebuildnorthbay.org,to,0.7,2020,,"View this email in your browser Each year, CCL...",cclr.org;rebuildnorthbay.org,1,2.2000,1,1,1,0,0,1
2,12125,kim@swelltheagency.com,from,1.0,2025,Re:,"Jennifer, So kind of Tony to introduce us. An...",afterthefireusa.org;swelltheagency.com;teampal...,1,2.1655,0,1,1,1,0,1
3,12125,darragh@teampalisades.org,cc,0.4,2025,Re:,"Jennifer, So kind of Tony to introduce us. An...",afterthefireusa.org;swelltheagency.com;teampal...,1,2.1655,0,1,1,1,0,1
4,12125,jennifer@afterthefireusa.org,to,0.7,2025,Re:,"Jennifer, So kind of Tony to introduce us. An...",afterthefireusa.org;swelltheagency.com;teampal...,1,2.1655,0,1,1,1,0,1


In [ ]:
import json
import time
import math
import re
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential
from openai import OpenAI

client = OpenAI()

MODEL = "gpt-4.1-mini"   # fallback: gpt-4o-mini
BATCH_SIZE = 25
MAX_EVIDENCE = 30
SLEEP_BETWEEN_CALLS = 0.2


In [ ]:
def sheets_safe_values(df):
    rows = []
    for row in df.itertuples(index=False):
        clean_row = []
        for v in row:
            if isinstance(v, float) and (math.isnan(v) or math.isinf(v)):
                clean_row.append("")
            elif pd.isna(v):
                clean_row.append("")
            else:
                clean_row.append(v)
        rows.append(clean_row)
    return rows


In [ ]:
def email_to_domain(email):
    return email.split("@")[-1].lower().strip() if isinstance(email, str) and "@" in email else ""

def safe_float(x, default=0.0):
    if x is None:
        return default
    if isinstance(x, float) and (math.isnan(x) or math.isinf(x)):
        return default
    try:
        return float(x)
    except Exception:
        return default

def build_evidence_bundle(g):
    bundle = []
    for _, r in g.iterrows():
        bundle.append({
            "year": int(r["Year"]) if not pd.isna(r["Year"]) else None,
            "email_role": str(r["EmailRole"]) if not pd.isna(r["EmailRole"]) else "",
            "subject": str(r["Subject"]) if not pd.isna(r["Subject"]) else "",
            "excerpt": str(r["BodyExcerpt"])[:300] if not pd.isna(r["BodyExcerpt"]) else "",
            "domains": str(r["Domains"]) if not pd.isna(r["Domains"]) else "",
            "has_phone": bool(r["HasPhone"]) if not pd.isna(r["HasPhone"]) else False,
        })
    return bundle[:MAX_EVIDENCE]

def df_json_safe(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype.kind in ("f", "i"):
            df[col] = df[col].apply(
                lambda x: None if pd.isna(x) or math.isinf(x) else x
            )
        else:
            df[col] = df[col].apply(
                lambda x: "" if pd.isna(x) else x
            )
    return df


In [ ]:
def json_safe(obj):
    return json.loads(json.dumps(obj, allow_nan=False))

@retry(stop=stop_after_attempt(3), wait=wait_exponential(min=2, max=10))
def call_llm(prompt):
    payload = {
        "model": MODEL,
        "temperature": 0.2,
        "messages": [
            {
                "role": "system",
                "content": (
                    "You analyze professional email evidence.\n"
                    "Do NOT invent organization names.\n"
                    "Use email domain as organization identifier.\n"
                    "Return STRICT JSON only."
                )
            },
            {"role": "user", "content": str(prompt)}
        ]
    }

    payload = json_safe(payload)
    resp = client.chat.completions.create(**payload)
    return resp.choices[0].message.content


In [ ]:
def build_prompt(email, org_domain, metrics, evidence):
    return f"""
You are classifying a person based on email participation evidence.

IMPORTANT CONTEXT:
- AfterTheFireUSA (afterthefireusa.org) is the HOST organization.
- All evidence includes interaction with this nonprofit.
- Do NOT infer a person's sector or role (e.g. Nonprofit)
  solely because they interact with AfterTheFireUSA.
- Classify the person based on THEIR organization and role,
  not the nature of AfterTheFireUSA.

Hard facts:
- Email: {email}
- OrgDomain (the person's organization): {org_domain}
- InternalEmailShare: {metrics["InternalEmailShare"]}
- InternalEmailCount: {metrics["InternalEmailCount"]}
- ExternalEmailCount: {metrics["ExternalEmailCount"]}

Evidence (emails involving AfterTheFireUSA):
{json.dumps(evidence, indent=2)}


TASKS:
1. Determine RelationshipToATF (choose ONE):
   - Internal staff
   - External collaborator
   - Government liaison
   - Press / Media
   - Event participant
   - Other (brief)

2. Identify ContextRoles as weighted labels (0–1).
   Examples: Government, Policy/Federal, Media, Event/Conference,
   Academic, Insurance, Nonprofit, Industry.

3. If a professional role/title is explicitly presented, provide it.
   Otherwise return null.

4. Provide overall confidence (0–1).

5. Provide a short rationale (1–3 sentences).

OUTPUT STRICT JSON ONLY:
{{
  "RelationshipToATF": "...",
  "ContextRoles": {{"Label": weight}},
  "RoleTitle": "... or null",
  "Confidence": 0.0,
  "Rationale": "..."
}}
"""


In [ ]:
ISO_TS_RE = re.compile(r"^\d{4}-\d{2}-\d{2}T")

already_done = set(
    people_df.loc[
        people_df["AffiliationLastUpdated"]
        .astype(str)
        .str.match(ISO_TS_RE),
        "Email"
    ]
)

print("Already processed (strict):", len(already_done))


Already processed (strict): 0


In [ ]:
metrics_lookup = {
    r["Email"]: r
    for _, r in person_df.iterrows()
}

evidence_groups = {
    email: g
    for email, g in sampled_people_df.groupby("ParticipantEmail")
}


In [ ]:
updates = []

for idx, row in people_df.iterrows():
    email = str(row.get("Email", "")).strip().lower()

    if not email or "@" not in email:
        continue
    if email in already_done:
        continue
    if email not in evidence_groups:
        continue

    org_domain = email_to_domain(email)

    raw = metrics_lookup.get(email, {})
    metrics = {
        "InternalEmailShare": safe_float(raw.get("InternalEmailShare"), 0.0),
        "InternalEmailCount": int(raw.get("InternalEmailCount", 0) or 0),
        "ExternalEmailCount": int(raw.get("ExternalEmailCount", 0) or 0),
    }

    evidence = build_evidence_bundle(evidence_groups[email])
    prompt = build_prompt(email, org_domain, metrics, evidence)

    try:
        parsed = json.loads(call_llm(prompt))
    except Exception as e:
        print("❌ LLM failure for", email, e)
        continue

    updates.append({
        "row_index": idx,
        "OrgDomain": org_domain,
        "RelationshipToATF": parsed.get("RelationshipToATF"),
        "ContextRoles": json.dumps(parsed.get("ContextRoles", {})),
        "RoleTitle": parsed.get("RoleTitle"),
        "AffiliationConfidence": safe_float(parsed.get("Confidence"), 0.0),
        "AffiliationRationale": parsed.get("Rationale"),
        "AffiliationLastUpdated": pd.Timestamp.utcnow().isoformat()
    })

    # ---------- CHECKPOINT WRITE (JSON-SAFE) ----------
    if len(updates) >= BATCH_SIZE:
        for u in updates:
            for k, v in u.items():
                if k != "row_index":
                    people_df.at[u["row_index"], k] = v

        values = (
            [people_df.columns.tolist()] +
            sheets_safe_values(people_df)
        )

        # Hard fail HERE if anything is still unsafe
        json.dumps(values, allow_nan=False)

        people_ws.update(values)

        print(f"✅ Checkpoint saved ({len(updates)} rows)")
        updates = []

    time.sleep(SLEEP_BETWEEN_CALLS)


✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint saved (25 rows)
✅ Checkpoint s

In [ ]:
if updates:
    for u in updates:
        for k, v in u.items():
            if k != "row_index":
                people_df.at[u["row_index"], k] = v

    values = (
        [people_df.columns.tolist()] +
        sheets_safe_values(people_df)
    )

    # Hard fail HERE if anything is still unsafe
    json.dumps(values, allow_nan=False)

    people_ws.update(values)

print("🎉 BLOCK 7 COMPLETE — LLM affiliation + roles written safely")


🎉 BLOCK 7 COMPLETE — LLM affiliation + roles written safely


In [ ]:
if updates:
    for u in updates:
        for k, v in u.items():
            if k != "row_index":
                people_df.at[u["row_index"], k] = v

    people_ws.update(
        [people_df.columns.tolist()] +
        people_df.values.tolist()
    )

print("🎉 BLOCK 7 COMPLETE — LLM affiliation + roles written")


🎉 BLOCK 7 COMPLETE — LLM affiliation + roles written
